# grex feed
Configure and run this to submit new work to the grex tasks Celery machinery

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import neo4j
import grexutils as gu
from grex_tasks import add, run_epp_code_strings
import random

/home/jovyan/work/nbs/grex_tasks.py:11: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


In [3]:
driver = neo4j.GraphDatabase.driver("neo4j://neo4j:7687", auth=("neo4j", "test"))
driver.verify_connectivity()

<ipython-input-3-e75ba85668dd>:2: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


{IPv4Address(('neo4j', 7687)): [{'ttl': 300,
   'servers': [{'addresses': ['neo4j:7687'], 'role': 'WRITE'},
    {'addresses': ['neo4j:7687'], 'role': 'READ'},
    {'addresses': ['neo4j:7687'], 'role': 'ROUTE'}]}]}

In [4]:
ex_key = gu.get_experiment_key_from_name(driver, name='t4')
ex_key

'jtsgosW1bfzg3SdgkYLqSw'

In [5]:
procedures = dict(gu.get_procedure_names_keys_from_experiment_key(driver, key=ex_key))
procedures

{'Train Pauls ADCs': 'bhvWVUoZNgTbyhBUSNcAkg',
 'even longer Train ADCs': 'pS8wyygG1oc5og7pMAlVGQ',
 'even Train ADCs': 'fUPzmkqZjtdtFGiiIqGrGQ',
 'Train ADCs': 'jFCVjYTVbUZGvAJhUE-mtw'}

In [6]:
proc_key = procedures['Train Pauls ADCs']

In [7]:
proc_key

'bhvWVUoZNgTbyhBUSNcAkg'

In [8]:
param_keys = gu.get_unstarted_parameters_of_procedure(driver, procedure_unikey=proc_key)
param_keys[:3]

['IIkQO_igwkyXUrx7R31AUA', 'OIBCfT0UH_ODysEeTPiDeg', 'P0uZXOxBzpLg7w-VzOlJlQ']

In [9]:
random.shuffle(param_keys)

In [12]:
cnt = 0
for param_key in param_keys:
    #print(ex_key, proc_key, param_key)
    run_epp_code_strings.delay(experiment_unikey=ex_key,
                        procedure_unikey=proc_key,
                        parameters_unikey=param_key)
    cnt += 1
    if cnt % 100 == 0:
        print(cnt, ex_key, proc_key, param_key)

100 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg zZQMwqloCPaWyLAqGjeCmA
200 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg SoJKp2lokwMnfoDr93hHRA
300 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg AnwhWQXNJnWU8IH8DI_Zxw
400 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg Vo5dbAD4VDaV2XFldUl1OQ
500 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg auOHQtX1HvIsIDhzau2hAw
600 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg KCgLHK0SVaq0hPqEoWG8KQ
700 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg USejm1QQSgzmGxavv_CPdg
800 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg C-s69TwGmUEFOFxp-suAgQ
900 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg pYzblq0BKQNihr1vjPUKkA
1000 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg 9oQUdg8nT3Xd2-PbSINYMQ
1100 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg b0yoc-POhWPYfc7IPFJzpw
1200 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg JvtfFgD1yRT_bfGT2R8Wyw
1300 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNcAkg MxRcMBE1GeL3zKZAIELJCg
1400 jtsgosW1bfzg3SdgkYLqSw bhvWVUoZNgTbyhBUSNc

In [11]:
assert False, "stop here"

AssertionError: stop here

---

# Debugging scratch space

In [ ]:
gu.get_code_strings_of_experiment_procedure_parameters(driver,
                                                       experiment_unikey='7R42cwhPZvhpmEUd51W5xQ',
                                                       procedure_unikey='ryDcs8eBJWihUWOEXdEEfQ',
                                                       parameters_unikey='vPVIVIYh_9GuMqG-DoaEPg')

In [ ]:
import tools.neotools as nj
def get_code_strings_of_experiment_procedure_parameters(driver, **kwargs):
    q = """
MATCH (e:Experiment {unikey: $experiment_unikey})
-[:INCLUDES]->
(proc:Procedure {unikey: $procedure_unikey})
-[:INCORPORATES]->
(par:Parameters {unikey: $parameters_unikey})
RETURN par.prepend_code_strings, proc.code_strings, par.append_code_strings
"""
    records = nj.query_read_return_list(driver, q, **kwargs)
    if len(records) < 1:
        raise KeyError(f'No experiment,procedure,parameters found to match "{kwargs}"')
    if len(records) > 1:
        raise KeyError(f'Found {len(records)} experiment,procedure,parameters "{kwargs}"')
    r = records[0]
    #print(f"r['proc.code_strings'] = {r['proc.code_strings']}")
    #print(f"r['par.code_strings'] = {r['par.code_strings']}")
    return r['par.prepend_code_strings'] + \
            r['proc.code_strings'] + \
            r['par.append_code_strings']

In [ ]:
import tools.neotools as nj
def get_code_strings_of_experiment_procedure_parameters(driver, **kwargs):
    q = """
MATCH (e:Experiment {unikey: $experiment_unikey})
-->
(proc:Procedure {unikey: $procedure_unikey})
-->
(par:Parameters {unikey: $parameters_unikey})
RETURN par.prepend_code_strings, proc.code_strings, par.append_code_strings
"""
    records = nj.query_read_return_list(driver, q, **kwargs)
    if len(records) < 1:
        raise KeyError(f'No experiment,procedure,parameters found to match "{kwargs}"')
    if len(records) > 1:
        raise KeyError(f'Found {len(records)} experiment,procedure,parameters "{kwargs}"')
    r = records[0]
    #print(f"r['proc.code_strings'] = {r['proc.code_strings']}")
    #print(f"r['par.code_strings'] = {r['par.code_strings']}")
    return r['par.prepend_code_strings'] + \
            r['proc.code_strings'] + \
            r['par.append_code_strings']

In [ ]:
import tools.neotools as nj
def get_code_strings_of_experiment_procedure_parameters(driver, **kwargs):
    q = """
MATCH (e:Experiment {unikey: $experiment_unikey})
-->
(proc:Procedure {unikey: $procedure_unikey})
RETURN proc.code_strings
"""
    records = nj.query_read_return_list(driver, q, **kwargs)
    if len(records) < 1:
        raise KeyError(f'No experiment,procedure,parameters found to match "{kwargs}"')
    if len(records) > 1:
        raise KeyError(f'Found {len(records)} experiment,procedure,parameters "{kwargs}"')
    r = records[0]
    #print(f"r['proc.code_strings'] = {r['proc.code_strings']}")
    #print(f"r['par.code_strings'] = {r['par.code_strings']}")
    return r

In [ ]:
import tools.neotools as nj
def get_code_strings_of_experiment_procedure_parameters(driver, **kwargs):
    q = """
MATCH (par:Parameters {unikey: $parameters_unikey})
RETURN par.prepend_code_strings
"""
    records = nj.query_read_return_list(driver, q, **kwargs)
    if len(records) < 1:
        raise KeyError(f'No experiment,procedure,parameters found to match "{kwargs}"')
    if len(records) > 1:
        raise KeyError(f'Found {len(records)} experiment,procedure,parameters "{kwargs}"')
    r = records[0]
    #print(f"r['proc.code_strings'] = {r['proc.code_strings']}")
    #print(f"r['par.code_strings'] = {r['par.code_strings']}")
    return r

In [ ]:
get_code_strings_of_experiment_procedure_parameters(driver,
                                                       experiment_unikey='7R42cwhPZvhpmEUd51W5xQ',
                                                       procedure_unikey='ryDcs8eBJWihUWOEXdEEfQ',
                                                       parameters_unikey='vPVIVIYh_9GuMqG-DoaEPg')

In [ ]:
for i in range(10):
    for j in range(10):
        add.apply_async((i,j), countdown=3*j)

In [ ]:
from tasks import div

In [ ]:
div.delay(1, 7)

In [ ]:
div.delay(22, 7)

In [ ]:
div.delay(1,0)